In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from KNN_D import KNN_D


In [2]:
df_host = pd.read_csv('host_data.csv')
df_loc = pd.read_csv('location_data.csv')
df_pric = pd.read_csv('pricing_data.csv')
df_prop = pd.read_csv('property_data.csv')
df_rev = pd.read_csv('review_data.csv')

In [3]:
dfs = [df_host, df_loc, df_pric, df_prop, df_rev]
for i in dfs:
    print(i.shape)
    print(i.head(1))

(69546, 5)
                                         customer_id  host_since  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...  10/07/2013   

  host_response_rate host_has_profile_pic host_identity_verified  
0               100%                    t                      t  
(69546, 6)
                                         customer_id     neighbourhood  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...  Columbia Heights   

    latitude  longitude city zipcode  
0  38.921768 -77.030985   DC   20009  
(69546, 4)
                                         customer_id  log_price  cleaning_fee  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...        4.0         False   

  cancellation_policy  
0            moderate  
(69546, 7)
                                         customer_id property_type  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...        house    

        room_type  bedrooms  bathrooms  accommodates  beds  
0   private room        1.0        1.0       

In [4]:
for i, df in enumerate(dfs, start=1):
    num_duplicates = df.duplicated(subset=['customer_id'], keep=False).sum()
    print(f"DataFrame {i}: {num_duplicates} duplicate rows based on 'customer_id'")

DataFrame 1: 2728 duplicate rows based on 'customer_id'
DataFrame 2: 2728 duplicate rows based on 'customer_id'
DataFrame 3: 2728 duplicate rows based on 'customer_id'
DataFrame 4: 2728 duplicate rows based on 'customer_id'
DataFrame 5: 2728 duplicate rows based on 'customer_id'


In [5]:
for i, df in enumerate(dfs, start=1):
    num_duplicates = df.duplicated(keep=False).sum()
    print(f"DataFrame {i}: {num_duplicates} fully identical rows")

DataFrame 1: 2728 fully identical rows
DataFrame 2: 2728 fully identical rows
DataFrame 3: 2728 fully identical rows
DataFrame 4: 2728 fully identical rows
DataFrame 5: 2728 fully identical rows


In [6]:

df_host = df_host.drop_duplicates(keep='first')
df_loc = df_loc.drop_duplicates(keep='first')
df_pric = df_pric.drop_duplicates(keep='first')
df_prop = df_prop.drop_duplicates(keep='first')
df_rev = df_rev.drop_duplicates(keep='first')
dfs = [df_host, df_loc, df_pric, df_prop, df_rev]


In [7]:
for i in dfs:
    print(i.shape)
    print(i.head(1))

(68182, 5)
                                         customer_id  host_since  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...  10/07/2013   

  host_response_rate host_has_profile_pic host_identity_verified  
0               100%                    t                      t  
(68182, 6)
                                         customer_id     neighbourhood  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...  Columbia Heights   

    latitude  longitude city zipcode  
0  38.921768 -77.030985   DC   20009  
(68182, 4)
                                         customer_id  log_price  cleaning_fee  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...        4.0         False   

  cancellation_policy  
0            moderate  
(68182, 7)
                                         customer_id property_type  \
0  4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...        house    

        room_type  bedrooms  bathrooms  accommodates  beds  
0   private room        1.0        1.0       

In [8]:
df = df_host.merge(df_loc, on='customer_id', how='inner') \
                   .merge(df_pric, on='customer_id', how='inner') \
                   .merge(df_rev, on='customer_id', how='inner') \
                   .merge(df_prop, on='customer_id', how='inner')
df['host_since'] = pd.to_datetime(df['host_since'])
df['first_review'] = pd.to_datetime(df['first_review'])
df['last_review'] = pd.to_datetime(df['last_review'])


In [9]:
print(df.shape)
print(df.duplicated().sum())

(68182, 23)
0


In [10]:
df.describe()

,host_since,latitude,longitude,log_price,number_of_reviews,review_scores_rating,first_review,last_review,bedrooms,bathrooms,accommodates,beds
count,68016,68182.000000,68182.000000,68182.000000,68182.000000,52773.000000,53563,53599,68102.000000,68005.000000,68182.000000,68070.000000
mean,2015-09-21 18:49:56.612561664,38.451586,-92.380279,4.651185,41.664750,94.067156,2016-01-15 08:09:17.578925824,2018-05-15 00:52:50.745722880,1.243796,1.235711,3.156244,1.711165
min,2009-05-03 00:00:00,33.338905,-122.511500,0.120000,0.000000,20.000000,2008-11-17 00:00:00,2010-03-21 00:00:00,-1.000000,0.000000,1.000000,0.000000
25%,2014-06-23 00:00:00,34.128665,-118.342356,4.000000,2.000000,92.000000,2015-06-29 00:00:00,2018-03-09 00:00:00,1.000000,1.000000,2.000000,1.000000
50%,2015-11-30 00:00:00,40.662825,-76.996166,4.715120,12.000000,96.000000,2016-05-26 00:00:00,2018-06-28 00:00:00,1.000000,1.000000,2.000000,1.000000
75%,2017-02-23 00:00:00,40.746423,-73.954541,5.000000,46.000000,100.000000,2017-01-02 00:00:00,2018-11-09 00:00:00,1.000000,1.000000,4.000000,2.000000
max,2018-12-04 00:00:00,42.390437,-70.985047,7.718399,1210.000000,100.000000,2017-10-04 00:00:00,2018-12-05 00:00:00,10.000000,8.000000,16.000000,18.000000
std,NaN,3.079089,21.702499,0.781907,75.304918,7.850617,NaN,NaN,0.878027,0.583663,2.158653,1.254449


In [11]:
print(df.isna().sum())

customer_id                   0
host_since                  166
host_response_rate        16866
host_has_profile_pic        166
host_identity_verified      166
neighbourhood              9368
latitude                      0
longitude                     0
city                          0
zipcode                     876
log_price                     0
cleaning_fee                  0
cancellation_policy           0
number_of_reviews             0
review_scores_rating      15409
first_review              14619
last_review               14583
property_type                 0
room_type                     0
bedrooms                     80
bathrooms                   177
accommodates                  0
beds                        112
dtype: int64


In [12]:
df['host_response_rate'] = pd.to_numeric(df['host_response_rate'].str.rstrip('%'))
#Change host_since to a numeric value ( we calculate the duration)
df['host_since'] = ((pd.to_datetime(datetime.now()) - df['host_since']).dt.days) / 365
#Change t/f to True and False
df['host_has_profile_pic'] = df['host_has_profile_pic'].map({'t': True, 'f': False}).astype(bool)
df['host_identity_verified'] = df['host_identity_verified'].map({'t': True, 'f': False}).astype(bool)
#Capatalize values in room_type and property_type to have a uniform standard between values
df['room_type'] = df['room_type'].str.upper()
df['property_type'] = df['property_type'].str.upper()

In [13]:
df[df['host_since'] < 100]

,customer_id,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,city,zipcode,...,number_of_reviews,review_scores_rating,first_review,last_review,property_type,room_type,bedrooms,bathrooms,accommodates,beds
0,4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...,11.263014,100.0,True,True,Columbia Heights,38.921768,-77.030985,DC,20009,...,0,NaN,NaT,NaT,HOUSE,PRIVATE ROOM,1.0,1.0,2,1.0
1,XDBcz6zYzSF6AUGPCClRRhP6XjeHOHKCiybvgaevRF6J5Q...,10.506849,NaN,True,False,Westlake,34.057546,-118.288971,LA,90005,...,0,NaN,NaT,NaT,APARTMENT,ENTIRE HOME/APT,0.0,1.0,2,1.0
2,OMm373CPtgtCt3dep7huwYt2G9uGUOPhEG7JkE8ZAvKlMp...,8.208219,100.0,True,True,East New York,40.661924,-73.867641,NYC,11208.0,...,26,91.0,2017-01-01,2018-11-23,HOUSE,PRIVATE ROOM,1.0,1.0,2,1.0
3,OegBpWwCIpG3oc6jLqfYPpBRCCwGrcVh2xQsiUoZXSEtPp...,8.515068,NaN,True,True,South Pasadena,34.109652,-118.175605,LA,90042,...,0,NaN,NaT,NaT,VILLA,ENTIRE HOME/APT,1.0,1.0,3,1.0
4,837RAwLA6dEkJQHyGasNKdV3nZnunANTprjIo7J8C5wZL6...,12.273973,100.0,True,False,Upper East Side,40.766836,-73.959440,NYC,10021,...,64,98.0,2011-08-29,2018-09-28,APARTMENT,ENTIRE HOME/APT,1.0,1.0,2,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68177,f2BOUClwcSJFylWFm9ZZbmT33NN5zfrEC9acLFniZftsMG...,7.134247,100.0,True,True,Carroll Gardens,40.682925,-73.997469,NYC,11231.0,...,2,100.0,2017-08-06,2018-10-06,TOWNHOUSE,ENTIRE HOME/APT,5.0,2.5,12,8.0
68178,owkLdHeTXuNky7id8h0OCRjocypRtUlBklyBzCTgt0qc57...,8.616438,60.0,True,True,NaN,34.205908,-118.572788,LA,91306,...,12,100.0,2015-10-11,2018-03-30,HOUSE,PRIVATE ROOM,1.0,1.5,4,1.0
68179,rjooqKTDjUIHetuaCmvRgqsdCrwny9xJXw0aCMmB4xQUJp...,9.282192,97.0,True,True,Bushwick,40.689701,-73.906398,NYC,11207,...,0,NaN,NaT,NaT,NONE,PRIVATE ROOM,1.0,1.0,2,1.0
68180,HuiGLAnSeU06GTAp8IL4DUWz1fAKG9n3aU6K46MdGBssGz...,7.136986,100.0,True,False,Flatiron District,40.743522,-73.986369,NYC,10016,...,14,94.0,2017-07-08,2018-10-28,CONDOMINIUM,ENTIRE HOME/APT,2.0,1.0,6,3.0


In [14]:

# from geopy.geocoders import Nominatim
# from geopy.exc import GeocoderTimedOut


# geolocator = Nominatim(user_agent="lazaniaa7@gmail.com")

# def get_zipcode(lat, lon):
#     try:
#         location = geolocator.reverse((lat, lon), exactly_one=True, timeout=10)
#         if location and 'postcode' in location.raw['address']:
#             return location.raw['address']['postcode']
#     except GeocoderTimedOut:
#         return None
#     return None

# for idx, row in df[df['zipcode'].isna()].iterrows():
#     if pd.notna(row['latitude']) and pd.notna(row['longitude']):
#         zipcode = get_zipcode(row['latitude'], row['longitude'])
#         df.at[idx, 'zipcode'] = zipcode

# print(df.isna().sum())

In [15]:
# for i in df['zipcode'][df['neighbourhood'].isna()]:
#     try:
#         df.loc[(df['zipcode'] == i) & (df['neighbourhood'].isna()), 'neighbourhood'] = df['neighbourhood'][df['zipcode'] == i].mode()[0]
#     except:
#         df.loc[(df['zipcode'] == i) & (df['neighbourhood'].isna()), 'neighbourhood'] = 'NNF'
    
    
    

In [16]:
value_counts = df['city'].value_counts()
print(value_counts)

city
NYC        29798
LA         20604
SF          5940
DC          5201
Chicago     3442
Boston      3197
Name: count, dtype: int64


In [17]:
value_counts = df['room_type'].value_counts()
print(value_counts)

room_type
ENTIRE HOME/APT     37987
PRIVATE ROOM        28203
SHARED ROOM          1992
Name: count, dtype: int64


In [18]:
value_counts = df['property_type'].value_counts()
print(value_counts)

property_type
 APARTMENT              42839
 HOUSE                  14475
NONE                     3409
 CONDOMINIUM             2297
 TOWNHOUSE               1492
 LOFT                    1074
 OTHER                    525
 GUESTHOUSE               419
 BED & BREAKFAST          410
 BUNGALOW                 320
 VILLA                    155
 DORM                     116
 GUEST SUITE              106
 CAMPER/RV                 80
 HOSTEL                    64
 CABIN                     64
 IN-LAW                    63
 TIMESHARE                 62
 BOUTIQUE HOTEL            62
 BOAT                      59
 SERVICED APARTMENT        18
 TENT                      13
 CASTLE                    10
 VACATION HOME             10
 YURT                       9
 CHALET                     6
 TREEHOUSE                  6
 HUT                        6
 TIPI                       3
 EARTH HOUSE                3
 CAVE                       2
 PARKING SPACE              1
 CASA PARTICULAR          

In [19]:
value_counts = df['zipcode'].value_counts()
print(value_counts)

zipcode
11211.0    1246
90291      1163
11221      1102
94110       907
20002       878
           ... 
9004          1
92821         1
1m            1
10012.0       1
91377         1
Name: count, Length: 761, dtype: int64


In [20]:
filtered_df = df[df['zipcode'] == '1m']
filtered_df

,customer_id,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,city,zipcode,...,number_of_reviews,review_scores_rating,first_review,last_review,property_type,room_type,bedrooms,bathrooms,accommodates,beds
65893,wuPxnYtJlc7DnmfGA5mdbhVwHOFNX3I0wjRGR9i42kWQ60...,8.50411,100.0,True,False,Upper West Side,40.800009,-73.967564,NYC,1m,...,76,89.0,2015-08-07,2018-11-23,APARTMENT,PRIVATE ROOM,1.0,1.0,3,2.0


In [21]:
df.columns

Index(['customer_id', 'host_since', 'host_response_rate',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'latitude', 'longitude', 'city', 'zipcode', 'log_price', 'cleaning_fee',
       'cancellation_policy', 'number_of_reviews', 'review_scores_rating',
       'first_review', 'last_review', 'property_type', 'room_type', 'bedrooms',
       'bathrooms', 'accommodates', 'beds'],
      dtype='object')

In [22]:
df_prop

,customer_id,property_type,room_type,bedrooms,bathrooms,accommodates,beds
0,4mUYtiM8BCkYchSi0rELebTJxPM68aZdKyUkER7ol5HI32...,house,private room,1.0,1.0,2,1.0
1,XDBcz6zYzSF6AUGPCClRRhP6XjeHOHKCiybvgaevRF6J5Q...,apartment,entire home/apt,0.0,1.0,2,1.0
2,OMm373CPtgtCt3dep7huwYt2G9uGUOPhEG7JkE8ZAvKlMp...,HOUSE,private room,1.0,1.0,2,1.0
3,OegBpWwCIpG3oc6jLqfYPpBRCCwGrcVh2xQsiUoZXSEtPp...,villa,entire home/apt,1.0,1.0,3,1.0
4,837RAwLA6dEkJQHyGasNKdV3nZnunANTprjIo7J8C5wZL6...,apartment,entire home/apt,1.0,1.0,2,1.0
...,...,...,...,...,...,...,...
69541,f2BOUClwcSJFylWFm9ZZbmT33NN5zfrEC9acLFniZftsMG...,townhouse,ENTIRE HOME/APT,5.0,2.5,12,8.0
69542,owkLdHeTXuNky7id8h0OCRjocypRtUlBklyBzCTgt0qc57...,house,PRIVATE ROOM,1.0,1.5,4,1.0
69543,rjooqKTDjUIHetuaCmvRgqsdCrwny9xJXw0aCMmB4xQUJp...,NONE,PRIVATE ROOM,1.0,1.0,2,1.0
69544,HuiGLAnSeU06GTAp8IL4DUWz1fAKG9n3aU6K46MdGBssGz...,CONDOMINIUM,entire home/apt,2.0,1.0,6,3.0


In [23]:
df_prop.isna().sum()

customer_id        0
property_type      0
room_type          0
bedrooms          80
bathrooms        177
accommodates       0
beds             112
dtype: int64

In [24]:
value_counts = df['bedrooms'].value_counts()
print(value_counts)

bedrooms
 1.0     45319
 2.0     10357
 0.0      6113
 3.0      3931
 4.0      1219
-1.0       682
 5.0       324
 6.0        93
 7.0        33
 8.0        13
 10.0       10
 9.0         8
Name: count, dtype: int64


In [25]:
value_counts = df['beds'].value_counts()
print(value_counts)

beds
1.0     41535
2.0     15379
3.0      5919
4.0      2802
5.0      1193
6.0       621
7.0       194
8.0       175
10.0       81
9.0        65
16.0       33
12.0       27
11.0       21
13.0       10
15.0        6
0.0         4
14.0        4
18.0        1
Name: count, dtype: int64


In [26]:
value_counts = df['bathrooms'].value_counts()
print(value_counts)

bathrooms
1.0    53436
2.0     7312
1.5     3504
2.5     1433
3.0      978
3.5      402
4.0      264
0.5      198
0.0      181
4.5      105
5.0       67
8.0       40
5.5       39
6.0       22
7.0       10
6.5        9
7.5        5
Name: count, dtype: int64


In [27]:
filtered_df = df[df['bathrooms'] == 7.5]
filtered_df.tail()

,customer_id,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,city,zipcode,...,number_of_reviews,review_scores_rating,first_review,last_review,property_type,room_type,bedrooms,bathrooms,accommodates,beds
1094,RiBrOUt1ij0M6rSCt35nX4SznPben8DBFm82T4so26kVZv...,9.753425,100.0,True,True,NaN,34.036311,-118.797051,LA,90265,...,22,88.0,2014-05-20,2017-08-02,HOUSE,ENTIRE HOME/APT,7.0,7.5,12,7.0
2073,YZJ4wwlE4WalFyBCPoARNV9jvZvuIRdVK99MVvdjqR0D76...,11.120548,100.0,True,True,Topanga,34.123808,-118.628714,LA,90290,...,0,NaN,NaT,NaT,VILLA,ENTIRE HOME/APT,8.0,7.5,16,11.0
7058,XkL3yNeNQtYZcpPGEUgzl4fpT8H9q7aIjirLruRJ9Ir9x2...,8.265753,100.0,True,True,Laurel Canyon,34.104248,-118.379877,LA,90046,...,12,100.0,2016-03-23,2018-06-24,HOUSE,ENTIRE HOME/APT,5.0,7.5,10,5.0
23042,uz0xgpjljBKZZtUiY9C8OiCbetG1iRyRSJZnUbCfFNaOo0...,9.482192,100.0,True,True,Malibu,34.023665,-118.802350,LA,90265,...,2,100.0,2014-06-27,2015-08-27,HOUSE,ENTIRE HOME/APT,6.0,7.5,14,9.0
67114,RRwIJckw6dDmttOQhiLCPAltKH77Mc8si4kNPYmfry9PKr...,7.150685,100.0,True,False,NaN,34.118500,-118.662068,LA,91302,...,0,NaN,NaT,NaT,HOUSE,ENTIRE HOME/APT,6.0,7.5,16,6.0


In [28]:
filtered_df = df[df['bathrooms'] == 'NaN']
filtered_df.tail()

,customer_id,host_since,host_response_rate,host_has_profile_pic,host_identity_verified,neighbourhood,latitude,longitude,city,zipcode,...,number_of_reviews,review_scores_rating,first_review,last_review,property_type,room_type,bedrooms,bathrooms,accommodates,beds


In [29]:
df_prop['property_type'].value_counts()

property_type
 APARTMENT       21423
 apartment       21416
 HOUSE            7299
 house            7176
none              1732
                 ...  
 TIPI                1
 TRAIN               1
 earth house         1
 hut                 1
 LIGHTHOUSE          1
Name: count, Length: 67, dtype: int64

In [30]:
print(df_prop[df_prop['room_type'] == 'hut'])

Empty DataFrame
Columns: [customer_id, property_type, room_type, bedrooms, bathrooms, accommodates, beds]
Index: []


In [31]:
df_prop['bedrooms'] = df_prop['bedrooms'].dropna()
df_prop['beds'] = df_prop['beds'].dropna()
df_prop['bathrooms'] = df_prop['bathrooms'].dropna()

df_prop.isna().sum()

customer_id        0
property_type      0
room_type          0
bedrooms          80
bathrooms        177
accommodates       0
beds             112
dtype: int64

In [32]:
df_prop = df_prop.dropna(subset=['bedrooms', 'beds', 'bathrooms'])
df_prop.isna().sum()

customer_id      0
property_type    0
room_type        0
bedrooms         0
bathrooms        0
accommodates     0
beds             0
dtype: int64

In [33]:
df_prop['bedrooms'] = df['bedrooms'].replace(-1, 1)

C:\Users\Gamer\AppData\Local\Temp\ipykernel_43860\3573675190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prop['bedrooms'] = df['bedrooms'].replace(-1, 1)


In [34]:
#KNN model Bathrooms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

k = 5
knn = KNeighborsRegressor(n_neighbors=k)

df_prop = df_prop.dropna(subset=['bedrooms', 'beds', 'bathrooms'])

X = df_prop.drop(columns=['bathrooms','customer_id']) 
X = pd.get_dummies(X, columns=['property_type','room_type'], drop_first=True)
# X['bedrooms'] = X['bedrooms'].dropna()
# X['beds'] = X['beds'].dropna()
# X['bedrooms'] = X['bedrooms'].fillna(X['bedrooms'].mode()[0])
# X['beds'] = X['beds'].fillna(X['beds'].mode()[0])

y = df_prop['bathrooms']
# y = y.dropna()
# y = y.fillna(y.mode()[0])

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("KNN Regressor Metrics:")
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R² Score:", r2)


KNN Regressor Metrics:
Mean Squared Error (MSE): 0.25337210468264315
Mean Absolute Error (MAE): 0.26732176877569436
R² Score: 0.2757946445257229


In [35]:
#KNN model Bedrooms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

k=5
knn = KNeighborsClassifier(n_neighbors=k)

df_prop = df_prop.dropna(subset=['bedrooms', 'beds', 'bathrooms'])

X = df_prop.drop(columns=['bedrooms','customer_id']) 
X = pd.get_dummies(X, columns=['property_type','room_type'], drop_first=True)


y = df_prop['bedrooms']
# y = y.fillna(y.mode()[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("KNN Model Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

KNN Model Accuracy: 0.6273438283365086

Confusion Matrix:
 [[   86  1658    32    58     0     0     0     0     0     0     0]
 [  472 12327   227   483     0     0     0     0     0     0     0]
 [  115  2709    46   106     0     0     0     0     0     0     0]
 [   36  1038    19    54     0     0     0     0     0     0     0]
 [   11   320     7    16     0     0     0     0     0     0     0]
 [    5    74     2     4     0     0     0     0     0     0     0]
 [    2    22     0     0     0     0     0     0     0     0     0]
 [    0    10     0     0     0     0     0     0     0     0     0]
 [    1     3     0     0     0     0     0     0     0     0     0]
 [    0     1     0     0     0     0     0     0     0     0     0]
 [    0     1     0     1     0     0     0     0     0     0     0]]

Classification Report:
               precision    recall  f1-score   support

         0.0       0.12      0.05      0.07      1834
         1.0       0.68      0.91      0.78    

c:\Users\Gamer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gamer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gamer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [36]:
#KNN model Beds

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

k=5
knn = KNeighborsClassifier(n_neighbors=k)

df_prop = df_prop.dropna(subset=['bedrooms', 'beds', 'bathrooms'])

X = df_prop.drop(columns=['beds','customer_id']) 
X = pd.get_dummies(X, columns=['property_type','room_type'], drop_first=True)


y = df_prop['beds']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("KNN Model Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

KNN Model Accuracy: 0.7380427153313948

Confusion Matrix:
 [[    0     0     1     0     0     0     0     0     0     0     0     0
      0     0     0     0     0]
 [    0 10857  1159    44     6     5     4     1     2     0     1     0
      0     0     0     0     0]
 [    0  1311  2817   413    37     4     1     2     0     0     0     0
      0     0     0     0     0]
 [    0   130   724   734   150    26     3     0     0     0     0     0
      0     0     0     0     0]
 [    0    25   186   297   219    53    10     2     0     1     0     1
      0     0     0     0     0]
 [    0     3    41    97   109    59    23     5     4     1     0     0
      0     0     0     1     0]
 [    0     8    14    42    46    41    26     5     1     1     0     0
      0     0     0     0     0]
 [    0     1     4     5    16    14     8     2     3     1     1     0
      0     0     0     0     0]
 [    0     5     1     5    12     5    15     5     3     0     0     0
      1    

c:\Users\Gamer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gamer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Gamer\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [37]:
df_prop = df_prop.dropna(subset=['bedrooms', 'beds', 'bathrooms'])

X = df_prop.drop(columns=['beds','customer_id']) 
X = pd.get_dummies(X, columns=['property_type','room_type'], drop_first=True)


y = df_prop['beds']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model_KNN_D = KNN_D(dcalc='manhatten')
model_KNN_D.fit(X_train, y_train)


y_train_pred = model_KNN_D.predict(X_train)
y_test_pred = model_KNN_D.predict(X_test)


train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)

print(train_score)
print(test_score)


0.611887060036959
0.6055850797152311
